# Amplitude Amplification Module

The present notebook reviews the *amplitude_amplification* module from the *AA* package in the *QQuantLib* library (**QQuantLib/AA/amplitude_amplification.py**). This module implements all the necessary functions for constructing Grover-like operators, which are essential for **amplitude amplification** procedures.

This notebook and the associated module are based on the following references:

- Brassard, G., Hoyer, P., Mosca, M., & Tapp, A. (2000). Quantum Amplitude Amplification and Estimation. *AMS Contemporary Mathematics Series*, 305. [arXiv:quant-ph/0005055v1](https://arxiv.org/abs/quant-ph/0005055v1)
- NEASQC Deliverable: *D5.1: Review of State-of-the-Art for Pricing and Computation of VaR*. [PDF Link](https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf)

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
#This cell loads the QLM solver. See notebook: 00_AboutTheNotebooksAndQPUs.ipynb
from QQuantLib.qpu.get_qpu import get_qpu
# myqlm qpus: python, c
# QLM qpus accessed using Qaptiva Access library: qlmass_linalg, qlmass_mps
# QLM qpus: Only in local Quantum Learning Machine: linalg, mps
my_qpus = ["python", "c", "qlmass_linalg", "qlmass_mps", "linalg", "mps"]

linalg_qpu = get_qpu(my_qpus[1])

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results

## 1. Data loading

Before performing any amplitude amplification, we need to load data into the quantum circuit. As this step is auxiliary and intended only to demonstrate the effect of amplification, we will load a discrete probability distribution. In this case, we will use a circuit with $ n = 3 $ qubits, which results in a total of $ N = 2^n = 8 $ computational basis states. The discrete probability distribution we will load is defined as:
$$
p_d = \frac{(0, 1, 2, 3, 4, 5, 6, 7)}{0 + 1 + 2 + 3 + 4 + 5 + 6 + 7}.
$$
Note that this probability distribution is properly normalized.

To achieve this, we will use the `load_probability` function from **QQuantLib/DL/data_loading**. The resulting quantum state can be expressed as:
$$
|\Psi\rangle = \frac{1}{\sqrt{0 + 1 + 2 + 3 + 4 + 5 + 6 + 7}} \left( \sqrt{0}|0\rangle + \sqrt{1}|1\rangle + \sqrt{2}|2\rangle + \sqrt{3}|3\rangle + \sqrt{4}|4\rangle + \sqrt{5}|5\rangle + \sqrt{6}|6\rangle + \sqrt{7}|7\rangle \right).
$$

In [ ]:
from QQuantLib.DL.data_loading import load_probability, load_array

In [ ]:
n = 3
N = 2**n
x = np.arange(N)
p = x/np.sum(x)

In [ ]:
probability_routine = qlm.QRoutine()
register = probability_routine.new_wires(n)
probability_routine.apply(load_probability(p),register)
%qatdisplay probability_routine --svg

In [ ]:
results_loading, _, _, _ = get_results(probability_routine, linalg_qpu=linalg_qpu, shots=0)
amplitudes_loading = results_loading["Amplitude"].values
results_loading

In [ ]:
print("Classical probabilities: ", np.sqrt(p))
print("Quantum probabilities: ",amplitudes_loading)
print('Test OK?: ',np.isclose(np.sqrt(p), amplitudes_loading).all())

Observe that the information stored in the quantum state is equivalent to the data contained in the array $ p $. For further details on how to load data into a quantum circuit, refer to the notebook *01_Data_Loading_Module_Use.ipynb*.

## 2. Reflections

**Reflections** are essential components for amplitude amplification. A reflection effectively rotates the phase of a quantum state by $ \frac{\pi}{2} $. In simpler terms, for real-valued amplitudes, **it changes the sign of the state**. 

The function `reflection` from the **QQuantLib/AA/amplitude_amplification** module is used to create these reflections.

In [ ]:
from QQuantLib.AA.amplitude_amplification import reflection

Creating a reflection over a quantum state using the aforementioned function involves a two-step process:

1. **Creating the Reflection QLM Abstract Gate**  
   This is achieved by calling the `reflection` function and specifying the quantum state over which the reflection will be applied. The state must be provided as a Python list when the function is called. For example:
   ```python
   state = [1, 0, 0, 1]
   RG = reflection(state)
   ```
   The function generates a QLM AbstractGate that represents the reflection operation.
2. **Applying the Reflection QLM Abstract Gate**
    The resulting reflection gate can then be applied to a QLM Program or QRoutine by specifying the qubits it will act upon. For example:
    ```python
    qlm_routine.apply(RG, [qbit[0], qbit[1], qbit[2], ...])
    ```

Let us now explore some examples to illustrate this process.

### NOTE

By default, the `reflection` function uses the standard QLM implementation for the mandatory multi-controlled Z gate. If the argument `mcz_qlm` is set to `False`, a multiplexor-based version of the multi-controlled Z gate will be used instead

### 2.1 First example

We will begin with a simple example. In this case, our goal is to flip the sign of the state $ |7\rangle $. To achieve this, we will use the function `reflection`. This function accepts a list as a parameter, where the list specifies which combination of qubits will undergo reflection. Since the binary representation of the state $ |7\rangle $ is $ 111 $, we need to pass the argument $ [1, 1, 1] $ to the function.

In [ ]:
reflection1 = qlm.QRoutine()
reflection1_register = reflection1.new_wires(n)
reflection1.apply(probability_routine,reflection1_register)
#Step 1: Creating reflection QLM Abstract Gate
ReflectionGate = reflection([1,1,1])
#Comment before line and uncoment following for For multiplexor implementation of multi-controlled Z gate
#ReflectionGate = reflection([1,1,1], mcz_qlm=False)
%qatdisplay ReflectionGate --depth  --svg
#Step 2: Applying ReflectionGate to the affected qbits: In present case the reflection gate affects all the qbits of the system
reflection1.apply(ReflectionGate, reflection1_register)
%qatdisplay reflection1 --depth 1 --svg

Now we print the amplitudes stored in the quantum circuit against the ones stored in the array $p$ to see the differences in sign.

In [ ]:
results_reflection1, _, _, _ = get_results(reflection1, linalg_qpu=linalg_qpu, shots=0)
amplitudes_reflection1 = results_reflection1["Amplitude"].values
results_reflection1

In [ ]:
print("Classical amplitudes: ", np.sqrt(p))
print("Quantum amplitudes: ",amplitudes_reflection1)

In [ ]:
print('Test OK: ',
    np.isclose(np.sqrt(p)[:-1], amplitudes_reflection1[:-1]).all() and
    np.isclose(np.sqrt(p)[-1], -amplitudes_reflection1[-1]).all()
     )

As we see the last state has changed from $0.5$ to $-0.5$.

### 2.2 Second example

In this example, we tackle a more complex task: flipping the sign of all states that start with a `1`. These states are:
- $ 00\mathbf{1} \longrightarrow |1\rangle $
- $ 01\mathbf{1} \longrightarrow |3\rangle $
- $ 10\mathbf{1} \longrightarrow |5\rangle $
- $ 11\mathbf{1} \longrightarrow |7\rangle $

To achieve this, we again use the `reflection` function. However, this time we apply the reflection operation only to the first qubit, as it determines whether the state starts with a `1`. By targeting the first register, we ensure that the sign flip is applied to all states where the first qubit is in the state $ |1\rangle $.

In [ ]:
reflection2 = qlm.QRoutine()
reflection2_register = reflection2.new_wires(n)
reflection2.apply(probability_routine,reflection2_register)
#Step 1: Creating reflection QLM Abstract Gate
ReflectionGate2 = reflection([1])
#Comment before line and uncomment following for For multiplexor implementation of multi-controlled Z gate
#ReflectionGate2 = reflection([1], mcz_qlm=False) 
%qatdisplay ReflectionGate2 --depth   --svg
#Step 2: Applying ReflectionGate to the affected qubits: In this case the first qubit
reflection2.apply(ReflectionGate2,reflection2_register[0])
%qatdisplay reflection2 --depth 1 --svg

Now we will see that the positions $1,3,5$ and $7$ have a different sign from the ones in array $p_d$.

In [ ]:
results_reflection2, _, _, _ = get_results(reflection2, linalg_qpu=linalg_qpu, shots=0)
amplitudes_reflection2 = results_reflection2["Amplitude"].values
results_reflection2

In [ ]:
print("Classical amplitudes: ", np.sqrt(p))
print("Quantum amplitudes: ",amplitudes_reflection2)

In [ ]:
print('Test OK: ',
    np.isclose(
        np.sqrt(p)[[0, 2, 4, 6]],
        amplitudes_reflection2[[0, 2, 4, 6]]
    ).all() and
    np.isclose(
        np.sqrt(p)[[1, 3, 5, 7]],
        -amplitudes_reflection2[[1, 3, 5, 7]]
    ).all()
)

### 2.3 Third example

In this example, we will perform a reflection over a more specific set of states: $ |1\rangle, |3\rangle, |5\rangle, |6\rangle, |7\rangle $. To achieve this, we can break the process into two sub-reflections:

1. **Reflection over $ |1\rangle, |3\rangle, |5\rangle, |7\rangle $:**  
   This reflection was already implemented in **Section 2.2**, where the sign of all states starting with `1` (i.e., $ |1\rangle, |3\rangle, |5\rangle, |7\rangle $) was flipped.

2. **Reflection over $ |6\rangle $:**  
   The remaining state $ |6\rangle $ can be reflected using the same strategy outlined in **Section 2.1**, where a reflection is applied to a single target state.

By combining these two sub-reflections, we can effectively perform the desired reflection over the specified set of states.

In [ ]:
reflection3 = qlm.QRoutine()
reflection3_register = reflection3.new_wires(n)
reflection3.apply(probability_routine,reflection3_register)
reflection3.apply(reflection([1]),reflection3_register[0])
#Comment before line and uncoment following for For multiplexor implementation of multi-controlled Z gate
#reflection3.apply(reflection([1], mcz_qlm=False),reflection3_register[0])
reflection3.apply(reflection([1,1,0]),reflection3_register)
#Comment before line and uncoment following for For multiplexor implementation of multi-controlled Z gate
#reflection3.apply(reflection([1,1,0], mcz_qlm=False),reflection3_register)
%qatdisplay reflection3 --depth 2 --svg

Now we will see that the positions $1,3,5,6$ and $7$ have a different sign from the ones in array $p_d$.

In [ ]:
results_reflection3, _, _, _ = get_results(reflection3, linalg_qpu=linalg_qpu, shots=0)
amplitudes_reflection3 = results_reflection3["Amplitude"].values
results_reflection3

In [ ]:
print("Classical amplitudes: ", np.sqrt(p))
print("Quantum amplitudes: ",amplitudes_reflection3)

In [ ]:
print('Test OK: ',
    np.isclose(
        amplitudes_reflection3[[1, 3, 5, 6, 7]],
        -np.sqrt(p)[[1, 3, 5, 6, 7]]
    ).all() and
    np.isclose(
        amplitudes_reflection3[[0, 2, 4]],
        np.sqrt(p)[[0, 2, 4]]
    ).all()
)

Note that doing consecutive reflections over sets is a commutative operation **when the sets are disjoint**.

## 3. Grover operator

In general, the Grover operator operates as follows. Suppose we have an operator (or routine) that performs the transformation:
$$
\mathcal{O}|0\rangle = |\Psi \rangle = \sin(\theta)|\Psi_0\rangle + \cos(\theta)|\Psi_1\rangle,
$$
where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states. The Grover operator $\mathcal{G}$ then applies the following transformation:
$$
|\Psi \rangle \longrightarrow \mathcal{G}^k|\Psi\rangle = \sin\left((2k+1)\theta\right)|\Psi_0\rangle + \cos\left((2k+1)\theta\right)|\Psi_1\rangle.
$$

This operator is commonly referred to as **amplitude amplification** in the literature, as it *increases* the probability of obtaining the state $|\Psi_0\rangle$. However, note that when the angle $(2k+1)\theta$ exceeds $\frac{\pi}{2}$, the probability begins to decrease instead of increasing. Thus, when referring to "amplification," keep in mind that it can have this diminishing effect.

The Grover operator $\mathcal{G}$ can be decomposed into two distinct operators:
$$
\mathcal{G} = \hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_{0}\rangle},
$$
where $\hat{U}_{|\Psi_{0}\rangle}$ and $\hat{U}_{|\Psi\rangle}$ are defined as:
$$
\hat{U}_{|\Psi_{0}\rangle} = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|,
$$
$$
\hat{U}_{|\Psi\rangle} = \hat{I} - 2|\Psi\rangle \langle \Psi|.
$$

In this section, we will review all the operators implemented for constructing the Grover operator.

### 3.2 Operator $\hat{U}_{|\Psi_{0}\rangle}$

The first mandatory operator is defined as:
$$
\hat{U}_{|\Psi_{0}\rangle} = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|.
$$

When this operator is applied to the state $|\Psi\rangle$, the result is:
$$
\hat{U}_{|\Psi_{0}\rangle} |\Psi\rangle = -\sin(\theta)|\Psi_{0}\rangle + \cos(\theta)|\Psi_{1}\rangle.
$$

Thus, the operator $\hat{U}_{|\Psi_{0}\rangle}$ represents a reflection of the state $|\Psi_{0}\rangle$ around the state $|\Psi_{1}\rangle$. Graphically, this can be visualized as follows:




<img src="images/Grover.svg" width="500">

### Implementation in **QQuantLib**

The function `create_u0_gate` from the **QQuantLib/AA/amplitude_amplification** module allows us to create an Abstract Gate that performs this operation. Similar to the reflection process, applying $\hat{U}_{|\Psi_{0}\rangle}$ to a quantum state involves a two-step process:

1. **Creating the $\hat{U}_{|\Psi_{0}\rangle}$ QLM AbstractGate**  
   This is achieved using the `create_u0_gate` function, with the following inputs:
   - The first argument is always the `oracle` (i.e., the QLM QRoutine or Program).
   - The second argument is a list containing the binary representation of the state whose sign we want to flip (the marked state, `target`).
   - The third argument is a list of the qubit registers over which the operation will be applied (`index`).

2. **Applying the $\hat{U}_{|\Psi_{0}\rangle}$ QLM AbstractGate**  
   Once the gate is created, it can be applied to the specified qubit registers.

This process closely mirrors the syntax used for reflections, as `create_u0_gate` is internally implemented using the `reflection` function.

---

### **NOTE**
By default, the standard QLM construction for the multi-controlled Z gate is used. If the argument `mcz_qlm=False` is passed, a multiplexor-based version of the multi-controlled Z gate will be used instead.

Next, we provide two examples to illustrate this process.

In [ ]:
from QQuantLib.AA.amplitude_amplification import create_u0_gate

#### 3.2.1 First example

In this first example, our goal is to mark the state $ |7\rangle $. Its binary representation is $ 111 $, and we want to apply the operation to the qubit registers `[0, 1, 2]`. Following the arguments outlined above, we call the `create_u0_gate` function with the following inputs:

- **First Input**: The `oracle`, which is the QLM Routine containing the loaded probability distribution.
- **Second Input**: The state (`target`) we want to mark, provided in binary representation: `[1, 1, 1]`.
- **Third Input**: The registers (`index`) over which we want to apply the operation: `[0, 1, 2]`.

This configuration ensures that the sign of the state $ |7\rangle $ is flipped while leaving the other states unaffected.

In [ ]:
routine_U0_1 = qlm.QRoutine()
register_U0_1 = routine_U0_1.new_wires(probability_routine.arity)
routine_U0_1.apply(probability_routine,register_U0_1)
#Creating U0_gate
U0_gate = create_u0_gate(
    probability_routine, #oracle
    [1,1,1], #marked state
    [0,1,2] #affected qbits
)
#Comment before 5 lines and uncoment following for For multiplexor implementation of multi-controlled Z gate
#U0_gate = create_u0_gate(probability_routine, [1,1,1], [0,1,2], mcz_qlm=False)
%qatdisplay U0_gate --depth  --svg
#Apply U0_gate
routine_U0_1.apply(U0_gate,register_U0_1)
%qatdisplay routine_U0_1 --depth 2 --svg

If we visualize the circuit with one additional layer of depth, we will observe that it is implemented as a `reflection([1, 1, 1])` acting on the qubit registers `[0, 1, 2]`.

Finally, by examining the amplitudes stored in the quantum circuit, we can confirm that only the last element (corresponding to the state $ |7\rangle $) is affected by the operation.

In [ ]:
results_U0_1,_ ,_ ,_  = get_results(routine_U0_1,linalg_qpu)

In [ ]:
amplitudes_U0_1 = results_U0_1["Amplitude"].values
print("Classical probabilities: ", np.sqrt(p))
print("Quantum probabilities: ",amplitudes_U0_1)

In [ ]:
print('Test OK: ',
    np.isclose(
        np.sqrt(p)[:-1],
        amplitudes_U0_1[:-1]
    ).all() and
    np.isclose(
        np.sqrt(p)[-1],
        -amplitudes_U0_1[-1]
    ).all()
)

#### 3.2.1 Second example

In the second example, our goal is to flip the sign (or "mark") all states that begin with the digit `1`. These states are:
- $ 00\mathbf{1} \longrightarrow |1\rangle $,  
- $ 01\mathbf{1} \longrightarrow |3\rangle $,  
- $ 10\mathbf{1} \longrightarrow |5\rangle $, and  
- $ 11\mathbf{1} \longrightarrow |7\rangle $.  

To accomplish this, we will use the `create_u0_gate` function again, but this time we specify that the operation acts only on the first qubit register. The arguments passed to the `U0` function will be as follows:

- **First Input**: The oracle (`oracle`), which is the QLM Routine containing the loaded probability distribution.
- **Second Input**: The state we want to mark (`taget`), represented in binary form: `[1]`.
- **Third Input**: The registers (`index`) over which the operation will act: `[0]`.

This configuration ensures that the sign of all states starting with `1` is flipped while leaving the other states unaffected.

In [ ]:
routine_U0_2 = qlm.QRoutine()
register_U0_2 = routine_U0_2.new_wires(probability_routine.arity)
routine_U0_2.apply(probability_routine,register_U0_1)
#Creating U0_gate
U0_2_gate = create_u0_gate(
    probability_routine, #oracle
    [1], #marked state
    [0] #affected qbits
)
#Comment before 5 lines and uncomment following for For multiplexor implementation of multi-controlled Z gate
#U0_2_gate = create_u0_gate(probability_routine, [1], [0], mcz_qlm=False) 
%qatdisplay U0_2_gate --depth 0 --svg
routine_U0_2.apply(U0_2_gate,register_U0_2)
#Apply U0_gate
%qatdisplay routine_U0_2 --depth 1 --svg

If we display the circuit with one more layer of depth, we will see that it is implemented as a `reflection([1])` acting on the register `[0]`.

Last, we will show the amplitudes stored in the quantum circuit and we will see that only states $|1\rangle$, $|3\rangle$, $|5\rangle$ and $|7\rangle$ are affected.

In [ ]:
results_U0_2,_ ,_ ,_ = get_results(routine_U0_2,linalg_qpu)

In [ ]:
amplitudes_U0_2= results_U0_2["Amplitude"].values
print("Classical probabilities: ", np.sqrt(p))
print("Quantum probabilities: ",amplitudes_U0_2)

In [ ]:
print('Test OK: ',
    np.isclose(
        np.sqrt(p)[[0, 2, 4, 6]],
        amplitudes_U0_2[[0, 2, 4, 6]]
    ).all() and
    np.isclose(
        np.sqrt(p)[[1, 3, 5, 7]],
        -amplitudes_U0_2[[1, 3, 5, 7]]
    ).all()
)

**Note that we cannot use operator `create_u0_gate` to mark states that require more than one reflection. This is the case for example of the set of states $|1\rangle,|3\rangle,|5\rangle,|6\rangle,|7\rangle$.**

 ### 3.2 Operator $\hat{U}_{|\Psi}\rangle$

The **diffusion operator** $\hat{U}_{|\Psi\rangle}$ is defined as:

$$
\hat{U}_{|\Psi\rangle} = \hat{I} - 2 |\Psi\rangle\langle\Psi|,
$$

which represents a reflection of the state $|\Psi\rangle$ around the orthogonal state $|\Psi\rangle^{\perp}$, where $|\Psi\rangle^{\perp} \perp |\Psi\rangle$.

Using the graphical representation, we can see that:


<img src="images/StateReflection.svg" width="700">


$$
\hat{U}_{|\Psi\rangle} |\Psi\rangle = \hat{I} |\Psi\rangle - 2 |\Psi\rangle \langle\Psi|\Psi\rangle = |\Psi\rangle - 2 |\Psi\rangle = -|\Psi\rangle.
$$

This shows that the diffusion operator inverts the amplitude of the state $|\Psi\rangle$.

For implementing this operator, the function `create_u_gate` from the **QQuantLib/AA/amplitude_amplification** module will be used. This function requires the oracle as input.

#### Note:
By default, the QLM (Quantum Learning Machine) construction for the multi-controlled Z gate is used. If the parameter `mcz_qlm=False` is passed, a multiplexor-based version of the multi-controlled Z gate will be utilized instead.

In [ ]:
from QQuantLib.AA.amplitude_amplification import create_u_gate

In [ ]:
routine_U = qlm.QRoutine()
register_U = routine_U.new_wires(probability_routine.arity)
routine_U.apply(probability_routine,register_U)
routine_U.apply(create_u_gate(probability_routine),register_U) 
#Comment before line and uncomment following for For multiplexor implementation of multi-controlled Z gate
#routine_U.apply(create_u_gate(probability_routine, mcz_qlm=False),register_U)
%qatdisplay routine_U --svg --depth 2

When we show the results we will see that all the amplitudes have changed their sign.

In [ ]:
results_U,_ ,_ ,_ = get_results(routine_U,linalg_qpu)

In [ ]:
amplitudes_U = results_U["Amplitude"].values
print("Classical probabilities: ", np.sqrt(p))
print("Quantum probabilities: ",amplitudes_U)

In [ ]:
print('Test OK: ', np.isclose(np.sqrt(p), -amplitudes_U).all())

### 3.3 Grover

Now we have all the ingredients to create a Grover operator:

$$\hat{Q}=\hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_{0}\rangle}$$

The Grover operator can be implemented by function *grover* from **QQuantLib/AA/amplitude_amplification**. Let's see some examples

**NOTE**
By default, the QLM construction for the multi-controlled Z gate is used. If we pass **mcz_qlm=False** a multiplexor version of the multi-controlled Z gate will be used.

Now that we have all the necessary components, we can define the **Grover operator** as:

$$
\hat{Q} = \hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_0\rangle},
$$

where $\hat{U}_{|\Psi\rangle}$ is the diffusion operator and $\hat{U}_{|\Psi_0\rangle}$ is the oracle reflection operator. The Grover operator combines these two operations to amplify the amplitude of the desired state in quantum search algorithms.

The Grover operator can be implemented using the function `grover` from the **QQuantLib/AA/amplitude_amplification** module. Let's explore some examples.

#### Note:
By default, the QLM (Quantum Learning Machine) construction for the multi-controlled Z gate is used. If the parameter `mcz_qlm=False` is passed, a multiplexor-based version of the multi-controlled Z gate will be utilized instead.

In [ ]:
from QQuantLib.AA.amplitude_amplification import grover

As in the case of the $\hat{U}_{|\Psi_{0}\rangle}$ operator, the application of the Grover operator to a quantum state will be a two steps process:

1. **Creating the Grover QLM AbstractGate** using the *grover* and providing it, as input, the state that we want to amplify. This is done with the following three arguments:
    * The first one is always the oracle (this is the QLM QRoutine of the Program)
    * The second one is a list containing the binary representation of the state that we want to amplify (marked state).
    * The third one is a list of the registers over which we want to act. 
2. **Applying Grover QLM AbstractGate** to the registers that will be affected.



Similar to the $\hat{U}_{|\Psi_0\rangle}$ operator, applying the Grover operator to a quantum state involves a two-step process:

1. **Creating the Grover QLM AbstractGate** using the `grover` function and providing it with the necessary inputs to define the state we want to amplify. This requires the following three arguments:
   - **Oracle (QLM QRoutine):** The first argument is always the oracle, which is represented as a QLM QRoutine in the program (`oracle`).
   - **Marked State (Binary Representation):** The second argument is a list containing the binary representation of the state that we aim to amplify (the marked state, `target`).
   - **Target Registers:** The third argument is a list of the registers over which the Grover operator will act (`index`)

2. **Applying the Grover QLM AbstractGate** to the specified registers. Once the AbstractGate is created, it is applied to the registers that will be affected by the operation.

This two-step process ensures that the Grover operator is correctly configured and applied to achieve amplitude amplification in quantum search algorithms.

### 3.3.1 First example

In our first example we are going to amplify the probability of obtaining the state $|1\rangle$.

In [ ]:
grover1 = qlm.QRoutine()
grover1_register = grover1.new_wires(n)
grover1.apply(probability_routine,grover1_register)
#Creating Grover_Gate_gate
Grover_Gate = grover(
    probability_routine, #oracle
    [0,0,1], #marked state
    [0,1,2] #affected qubits
)
#Comment before 5 lines and uncomment following for For multiplexor implementation of multi-controlled Z gate
#Grover_Gate = grover(probability_routine, [0,0,1], [0,1,2], mcz_qlm=False)
%qatdisplay Grover_Gate --depth 2 --svg
#Applying the Grover Gate
grover1.apply(Grover_Gate, grover1_register)
%qatdisplay grover1 --depth 3 --svg

The critical part is *grover(probability_routine,[0,0,1],[0,1,2])*.
The first argument of the function is always the routine that we are going to amplify. 
In the second argument, we have to specify our target state for amplification. In this case, as we want to amplify the state $|1\rangle$ we use its binary representation $001$.
Last, the third input is an index which specifies over which part of the quantum register we want to act. In this case, as the three quantum registers are involved, we put the three of them $[0,1,2]$. In the next examples, it will become clearer how it works.

Now we measure the amplified probabilities

The critical part of the process is the function call:

```python
grover(probability_routine, [0, 0, 1], [0, 1, 2])
```

- The first argument of the function is always the routine that corresponds to the `oracle`, which identifies the state we want to amplify. In this case, it is probability_routine.
- The second argument specifies the `target` state that we aim to amplify. Since we want to amplify the state ∣1⟩, we provide its binary representation, which is `[0,0,1]`.
- The third argument is an `index` list that specifies the subset of the quantum register over which the Grover operator will act. Here, since all three qubits in the quantum register are involved, we include all three indices: `[0,1,2]`.

In the following examples, the functionality of these arguments will become clearer.

Now, let's proceed to measure the amplified probabilities.

In [ ]:
results_grover1,_ ,_ ,_ = get_results(grover1,linalg_qpu)
probabilities_grover1 = results_grover1["Probability"].values
probabilities_grover1

Let's summarize what we have done so far. In our example, we make the following identifications:

$$
\begin{aligned}
& \mathcal{O} \longrightarrow \mathcal{P}, \\
& |\Psi\rangle \longrightarrow \frac{1}{\sqrt{0+1+2+3+4+5+6+7+8}} \left[ \sqrt{0}|0\rangle + \sqrt{1}|1\rangle + \sqrt{2}|2\rangle + \sqrt{3}|3\rangle + \sqrt{4}|4\rangle + \sqrt{5}|5\rangle + \sqrt{6}|6\rangle + \sqrt{7}|7\rangle \right], \\
& \sin(\theta)|\Psi_0\rangle \longrightarrow \frac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}} |1\rangle, \\
& \cos(\theta)|\Psi_1\rangle \longrightarrow \frac{1}{\sqrt{0+1+2+3+4+5+6+7+8}} \left[ \sqrt{0}|0\rangle + \sqrt{2}|2\rangle + \sqrt{3}|3\rangle + \sqrt{4}|4\rangle + \sqrt{5}|5\rangle + \sqrt{6}|6\rangle + \sqrt{7}|7\rangle \right].
\end{aligned}
$$

After performing the amplification and measuring the probabilities, we obtain the new probability $ p_a = \sin^2(3\theta) $ for the state in position $ |1\rangle $. The original probability was $ p_o = \sin^2(\theta) $.

To verify that the amplified probability and the original probability are computed correctly, we can recover $\theta$ from the amplified probability using the formula:
$$
\theta = \frac{\arcsin\left(\sqrt{p_a}\right)}{3}.
$$

Substituting this angle back into the expression for the original probability, we get:
$$
p_o = \sin^2\left(\frac{\arcsin\left(\sqrt{p_a}\right)}{3}\right).
$$

In the next cell, we will confirm that the amplification process has been carried out correctly.

In [ ]:
unamplified_grover1 = np.sin(np.arcsin(np.sqrt(probabilities_grover1[1]))/3)**2
print("Original probabilities: ", p[1])
print("Quantum probabilities: ",unamplified_grover1)

### 3.2 Second example

In this second example we are going to show how to amplify a more complex state. Say that now we want to amplify the state composed by $|0\rangle,|1\rangle,|2\rangle,|3\rangle$. Now we will see the correspondence of this new setup with the Grover setup:
$$
    \begin{array}{l}
    &\mathcal{O}\longrightarrow \mathcal{P}.\\
    & |\psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & \sin(\theta)|\phi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle\right].\\
    & \cos(\theta)|\phi^\dagger\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$
In binary representation, the states $|0\rangle,|2\rangle,|4\rangle,|6\rangle$ are $|\mathbf{0}00\rangle,|\mathbf{0}01\rangle,|\mathbf{0}10\rangle,|\mathbf{0}11\rangle$ respectively. Here we are interested in the join probability of getting these states, this is the same as asking for the probability of the leftmost qubit being zero.
Before doing an amplification we are going to do the unamplified version.

In this second example, we will demonstrate how to amplify a more complex state. Specifically, we aim to amplify the state composed of $|0\rangle$, $|1\rangle$, $|2\rangle$, and $|3\rangle$. Below, we outline the correspondence of this new setup with the Grover framework:

$$
\begin{aligned}
& \mathcal{O} \longrightarrow \mathcal{P}, \\
& |\psi\rangle \longrightarrow \frac{1}{\sqrt{0+1+2+3+4+5+6+7+8}} \left[ \sqrt{0}|0\rangle + \sqrt{1}|1\rangle + \sqrt{2}|2\rangle + \sqrt{3}|3\rangle + \sqrt{4}|4\rangle + \sqrt{5}|5\rangle + \sqrt{6}|6\rangle + \sqrt{7}|7\rangle \right], \\
& \sin(\theta)|\phi\rangle \longrightarrow \frac{1}{\sqrt{0+1+2+3+4+5+6+7+8}} \left[ \sqrt{0}|0\rangle + \sqrt{1}|1\rangle + \sqrt{2}|2\rangle + \sqrt{3}|3\rangle \right], \\
& \cos(\theta)|\phi^\dagger\rangle \longrightarrow \frac{1}{\sqrt{0+1+2+3+4+5+6+7+8}} \left[ \sqrt{4}|4\rangle + \sqrt{5}|5\rangle + \sqrt{6}|6\rangle + \sqrt{7}|7\rangle \right].
\end{aligned}
$$

In binary representation, the states $|0\rangle$, $|1\rangle$, $|2\rangle$, and $|3\rangle$ correspond to $|\mathbf{0}00\rangle$, $|\mathbf{0}01\rangle$, $|\mathbf{0}10\rangle$, and $|\mathbf{0}11\rangle$, respectively. Here, we are interested in the joint probability of obtaining these states, which is equivalent to asking for the probability that the leftmost qubit is in the state $|0\rangle$.

Before proceeding with the amplification process, we will first compute the unamplified version of the probabilities.

In [ ]:
grover2 = qlm.QRoutine()
grover2_register = grover2.new_wires(n)
grover2.apply(probability_routine,grover2_register)
%qatdisplay grover2 --depth 0 --svg

In [ ]:
results_grover2,_,_,_  = get_results(grover2,linalg_qpu = linalg_qpu,qubits = [n-1])
results_grover2

When we measure only the leftmost qubit ($q_2$), we are effectively performing the following operation to compute the probabilities of its possible outcomes:

$$
\begin{aligned}
p_{|0\rangle} &= \sin^2(\theta) = \frac{1}{0+1+2+3+4+5+6+7+8} \left[ |\sqrt{0}|^2 + |\sqrt{1}|^2 + |\sqrt{2}|^2 + |\sqrt{3}|^2 \right], \\
p_{|1\rangle} &= \cos^2(\theta) = \frac{1}{0+1+2+3+4+5+6+7+8} \left[ |\sqrt{4}|^2 + |\sqrt{5}|^2 + |\sqrt{6}|^2 + |\sqrt{7}|^2 \right].
\end{aligned}
$$

Here, $p_{|0\rangle}$ represents the probability of measuring the state $|0\rangle$ for the leftmost qubit, while $p_{|1\rangle}$ corresponds to the probability of measuring the state $|1\rangle$. These probabilities are derived from the amplitudes of the respective subspaces.

In the next cell, we will verify these results by comparing them with the classical probability calculations.

In [ ]:
print(" Probability of state |0>: ", p[0]+p[1]+p[2]+p[3])
print(" Probability of state |1>: ", p[4]+p[5]+p[6]+p[7])

Next, we will amplify the probability of the state marked with the rightmost qubit being $0$. For that, we again use the *grover* function. Apart from the oracle we need to indicate the state that we want to amplify.

Next, we will amplify the probability of the state where the rightmost qubit is $ |0\rangle $. To achieve this, we once again utilize the `grover` function. In addition to providing the oracle, we need to specify the state that we aim to amplify.

Specifically:
- The `oracle` identifies the state(s) to be amplified.
- The `target` state is defined by the binary representation of the desired outcome. In this case, we focus on states where the rightmost qubit is $ |0\rangle $.
- The `index`refers to the qubit afected that in this case is $q_2$ (so `[2]`).

By applying the Grover operator, we enhance the probability amplitude of the target state, effectively increasing its likelihood of being measured.

In [ ]:
grover2 = qlm.QRoutine()
grover2_register = grover2.new_wires(n)
grover2.apply(probability_routine,grover2_register)
#Creating Grover_Gate_gate
Grover_Gate_2 = grover(
    probability_routine, #oracle
    [0], #marked state
    [n-1] #affected qbits
)
#Comment before 5 lines and uncoment following for For multiplexor implementation of multi-controlled Z gate
Grover_Gate_2 = grover(probability_routine, [0], [n-1], mcz_qlm=False)
%qatdisplay Grover_Gate_2 --depth 3 --svg
#Applying the Grover Gate
grover2.apply(Grover_Gate_2,grover2_register)
%qatdisplay grover2 --svg

Now we will measure again the probabilities of the leftmost qubit ($q_2$) being zero.

In [ ]:
results_grover2,_,_,_ = get_results(grover2,linalg_qpu = linalg_qpu,qubits = [n-1])
results_grover2

We clearly observe that the probability of obtaining $ |0\rangle $ has been increased, while the probability of obtaining $ |1\rangle $ has correspondingly decreased. This trade-off is a direct result of the amplification process performed by the Grover operator.

Moreover, we can establish a correspondence between the amplified probabilities and their unamplified counterparts. This comparison highlights how the Grover algorithm effectively enhances the likelihood of measuring the desired state, in this case, $ |0\rangle $, by redistributing the probability amplitudes.

In [ ]:
unamplified_grover2 = np.sin(np.arcsin(\
                    np.sqrt(results_grover2["Probability"].iloc[0]))/3)**2
print("Unamplified probability of state |0>: ", unamplified_grover2)
print("Classical probability of state |0>: ", p[0]+p[1]+p[2]+p[3])


### NOTE

In the module *utils* of package *utils* of the library *QQuantLib*  (**QQuantLib/utils/utils.py**) we have developed the *load_qn_gate* function that allows the use to create several applications of one given gate. We can use it for doing a multiple application of a Grover-like operator:


### Note

In the `utils` module of the **utils** package within the *QQuantLib* library (**QQuantLib/utils/utils.py**), we have implemented the `load_qn_gate` function, which enables users to create multiple applications of a given quantum gate. This functionality can be particularly useful for applying a Grover-like operator repeatedly in various quantum algorithms.

For instance, this function allows you to efficiently define and apply the Grover operator multiple times without having to reconstruct it from scratch each time, streamlining the implementation of amplitude amplification processes.

In [ ]:
from QQuantLib.utils.utils import load_qn_gate

In [ ]:
n_grover = load_qn_gate(grover(probability_routine,[0],[n-1]), 5)

In [ ]:
%qatdisplay n_grover --depth 1 --svg